<a href="https://colab.research.google.com/github/Juhwan01/DeepDive/blob/main/Bert_%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0_%EB%B6%84%EB%A5%98(%EC%A7%81%EC%A0%91_%EA%B5%AC%ED%98%84).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

In [2]:
# 네이버 영화 리뷰 데이터 학습을 위해 훈련 데이터와 테스트 데이터를 다운로드합니다.
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7de5b9b918d0>)

In [3]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [4]:
print('훈련용 리뷰 개수 :',len(train_data)) # 훈련용 리뷰 개수 출력
print('테스트용 리뷰 개수 :',len(test_data)) # 테스트용 리뷰 개수 출력

훈련용 리뷰 개수 : 150000
테스트용 리뷰 개수 : 50000


In [5]:
train_data.drop_duplicates(subset=['document'], inplace = True)
train_data = train_data.dropna(how = 'any')
print('훈련 데이터의 리뷰 수:',len(train_data))

훈련 데이터의 리뷰 수: 146182


In [6]:
test_data = test_data.dropna(how = 'any')
print('테스트 데이터의 리뷰 수:',len(test_data))

테스트 데이터의 리뷰 수: 49997


In [7]:
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [8]:
print(tokenizer.tokenize("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

['보', '##는', '##내', '##내', '그대로', '들어맞', '##는', '예측', '카리스마', '없', '##는', '악역']


In [9]:
print(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

[2, 1160, 2259, 2369, 2369, 4311, 20657, 2259, 5501, 13132, 1415, 2259, 23713, 3]


In [10]:
# 인코딩하면서 특수 토큰이 들어가고 다시 디코딩 하니 특수 토큰이 기존의 문장에서 추가되서 나옴(문장 복원 과정)
tokenizer.decode(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

'[CLS] 보는내내 그대로 들어맞는 예측 카리스마 없는 악역 [SEP]'

In [11]:
# 특수 토큰 이름과 ID 출력
print(tokenizer.cls_token,':', tokenizer.cls_token_id)
print(tokenizer.sep_token,':', tokenizer.sep_token_id)
print(tokenizer.pad_token,':', tokenizer.pad_token_id)

[CLS] : 2
[SEP] : 3
[PAD] : 0


In [12]:
max_length = 128

# 버전 차이로 인한 함수 변화
# encoded_result = tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화",max_seq_length=max_seq_length, pad_to_max_length=True)
# max_sequence_length -> max_length // pad_to_max_length=True -> padding = 'max_length'
encoded_result = tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화",max_length = max_length, padding='max_length')
print(encoded_result)
print('길이 :', len(encoded_result))

[2, 1537, 2534, 2069, 6572, 2259, 3771, 18, 3690, 4530, 2585, 2073, 3771, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
길이 : 128


In [13]:
print([0]*max_length)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [14]:
# 어텐션 마스크 인코딩
valid_num = len(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"))
print(valid_num * [1] + (max_length - valid_num) * [0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [15]:
def convert_examples_to_features(examples, labels, max_length, tokenizer):

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []
    # tqdm은 여기서 진행률 표시줄을 추가하기 위해서 사용한다
    # (examples, labels)로 묶고, examples의 길이만큼 반복한다 그걸 zip 객체를 감싸 진행바 표시
    # !!! truncation=True 옵션 추가 128 넘으면 자동으로 잘라 -> 원본 코드 돌려보니 128을 넘는 문장이 있기 때문
    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        input_id = tokenizer.encode(example, max_length=max_length, padding='max_length',truncation=True)

        # 단어 위치하면 1, 패딩은 0으로
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_length - padding_count) + [0] * padding_count

        # 세그먼트 인코딩 = 입력 [CLS] 나는 학생이다 [SEP] 너는 선생님이다 [SEP] ->
        #            Segment ID:   0     0     0       0    1       1        1
        token_type_id = [0] * max_length

        # assert -> 조건 만족하지 않으면 에러 발생시키는 디버킹 도구
        # input_id의 길이가 최대 시퀀스 길이와 같은지 확인
        # 만약 다르면 에러 메시지와 함께 실제 길이와 기대 길이를 출력
        assert len(input_id) == max_length, "Error with input length {} vs {}".format(len(input_id), max_length)

        # attention_mask의 길이가 최대 시퀀스 길이와 같은지 확인
        # attention_mask는 어떤 토큰이 실제 데이터이고 어떤 토큰이 패딩인지 표시
        assert len(attention_mask) == max_length, "Error with attention mask length {} vs {}".format(len(attention_mask), max_length)

        # token_type_id의 길이가 최대 시퀀스 길이와 같은지 확인
        # token_type_id는 BERT에서 첫 번째/두 번째 문장을 구분하는 데 사용 (0 또는 1)
        assert len(token_type_id) == max_length, "Error with token type length {} vs {}".format(len(token_type_id), max_length)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    # 리스트를 그대로 쓰면 파이썬 내장 자료구조라 연산 느리고 벡터연산도 불가능
    # -> 리스트+리스트 연결만되기 때문
    # 따라서 배열끼리 더하고 곱하는 것도 자연스러운 numpy 배열로 바꿔서 숫자를 효율적으로 처리하려고 변환을 한다
    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    # 라벨을 정수 배열로 변환해서 분류 문제에 적합하게 만듬
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels


In [16]:
train_X, train_y = convert_examples_to_features(train_data['document'],train_data['label'],max_length=max_length,tokenizer=tokenizer)

100%|██████████| 146182/146182 [00:44<00:00, 3251.46it/s]


In [17]:
test_X, test_y = convert_examples_to_features(test_data['document'],test_data['label'],max_length=max_length,tokenizer=tokenizer)

100%|██████████| 49997/49997 [00:15<00:00, 3331.41it/s]


In [18]:
input_id = train_X[0][0]
attention_mask = train_X[1][0]
token_type_id = train_X[2][0]
label = train_y[0]

print('단어에 대한 정수 인코딩 :', input_id)
print('어텐션 마스크 :', attention_mask)
print('세그먼트 인코딩 :', token_type_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :', tokenizer.decode(input_id))
print('레이블 :', label)


단어에 대한 정수 인코딩 : [   2 1376  831 2604   18   18 4229 9801 2075 2203 2182 4243    3    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
세그먼트 인코딩 : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [19]:
# PyTorch 형식으로 저장된 'klue/bert-base' 모델을
# TensorFlow 형식으로 변환해서 불러오기 위해 from_pt=True 옵션 사용
model = TFBertModel.from_pretrained("klue/bert-base",from_pt=True)

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'bert.embeddings.position_ids', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClas

In [20]:
max_length = 128

# 입력 토큰 ID들을 받는 Input 레이어 (배열 길이는 max_length, 데이터 타입은 32비트 정수)
input_ids_layer = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32)

# 어텐션 마스크를 받는 Input 레이어 (패딩 등에서 어느 토큰에 집중할지 표시, max_length 길이, int32)
attention_masks_layer = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32)

# 토큰 타입 ID를 받는 Input 레이어 (문장 A, B 구분용, max_length 길이, int32)
token_type_ids_layer = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32)

# BERT 모델에 정석 방식으로 입력-> 원본 입력은 첫번째 입력이 [input_ids_layer]로 감싸져 있어야함
outputs = model(
    input_ids=input_ids_layer,
    attention_mask=attention_masks_layer,
    token_type_ids=token_type_ids_layer
)

In [21]:
# 모든 토큰의 출력 shape: (batch_size, sequence_length, hidden_size) → 예: (32, 128, 768)
# 보면 128개의 출력이 나오는게 된다 -> 각출력 768차원 벡터 -> many to many
print(outputs[0])

KerasTensor(type_spec=TensorSpec(shape=(None, 128, 768), dtype=tf.float32, name=None), name='tf_bert_model/bert/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0', description="created by layer 'tf_bert_model'")


In [22]:
# 최종 768차원짜리 출력 하나 나옴 -> many to one이 된다.
print(outputs[1])

KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='tf_bert_model/bert/pooler/dense/Tanh:0', description="created by layer 'tf_bert_model'")


In [23]:
# tf.keras.model을 상속받아 사용 -> 상속받은 클래스의 compile,fit 같은 메소드를 사용할 수 있다.
class TFBertForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        # 파이썬3부터는 이렇게 간추려 쓸 수 있음 super(TFBertForSequenceClassification, self).__init__() -> super().__init__()
        super().__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        # 출력층을 하나 만들어서 classifier에 저장
        # 출력 뉴런 하나로 이진 분류 예측
        self.classifier = tf.keras.layers.Dense(1,
                                                # 가중치 초기화 방식
                                                # 정규분포에서 값을 뽑되 극단값은 버리는 버전
                                                # BERT 논문에 따르면, 모든 Dense 레이어는 TruncatedNormal(std=0.02)로 초기화해야 학습이 안정적이기 때문
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                # 이진분류 활성화 함수
                                                activation='sigmoid',
                                                name='classifier')
    def call(self, inputs):
        # 입력 받은 3개의 튜플 요소
        input_ids, attention_mask, token_type_ids = inputs
        # 모델에 입력값 전달
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # 이진분류 값을 받을거니 [1]번 선택
        # 문장의 대표 벡터인 [CLS] 토큰의 벡터 가져오기
        cls_token = outputs[1]
        # [CLS] 벡터를 Dense 레이어에 넣어서 최종 결과 예측
        # sigmoid이기 때문에 0~1사이의 확률
        prediction = self.classifier(cls_token)

        return prediction

In [24]:
model = TFBertForSequenceClassification("klue/bert-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'bert.embeddings.position_ids', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [25]:
optimizer = tf.keras.optimizers.Adam(5e-5)
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [26]:
model.fit(train_X, train_y, epochs=2, batch_size=64, validation_split=0.2)

Epoch 1/2
1828/1828 [==============================] - 388s 191ms/step - loss: 0.2834 - accuracy: 0.8801 - val_loss: 0.2516 - val_accuracy: 0.9013
Epoch 2/2
1828/1828 [==============================] - 339s 185ms/step - loss: 0.1898 - accuracy: 0.9256 - val_loss: 0.2383 - val_accuracy: 0.9011


In [27]:
results = model.evaluate(test_X,test_y, batch_size=1024)
print("test loss, test acc: ", results)

49/49 [==============================] - 36s 733ms/step - loss: 0.2503 - accuracy: 0.8949
test loss, test acc:  [0.25026610493659973, 0.8949136734008789]


In [29]:
def sentiment_predict(new_sentence):
  input_id = tokenizer.encode(new_sentence, max_length=max_length, padding='max_length',truncation=True)
  padding_count = input_id.count(tokenizer.pad_token_id)
  attention_mask = [1]*(max_length-padding_count) + [0]*padding_count
  token_type_id = [0]*max_length

  # 파이썬 리스트 구조 -> 처리 빠른 넘파이 배열 구조로
  input_ids = np.array([input_id])
  attention_masks = np.array([attention_mask])
  token_type_ids = np.array([token_type_id])

  encoded_input = [input_ids,attention_masks,token_type_ids]
  # model.predict는 배치 단위로 예측 결과를 반환함
  # many-to-one 문제라면 출력 형태는 (batch_size, output_dim)
  # many-to-many 문제라면 (batch_size, seq_len, output_dim) 형태임

  # 따라서 output[0]은 배치 첫 번째 샘플의 결과이고
  # output[0][0]은 (many-to-one일 때) 첫 번째 출력값 또는
  # (many-to-many일 때) 첫 번째 토큰의 출력값임

  # 이진 분류에서는 output_dim=1 이므로 output[0][0]으로 확률 값을 얻음
  # output은 (배치 크기, 시퀀스 길이, 출력 차원)의 3차원 배열임
  # output[0]은 배치의 첫 번째 샘플에 해당하는 시퀀스 전체 출력이고,
  # output[0][0]은 그 샘플 시퀀스 중 첫 번째 토큰에 대한 출력 벡터임
  # 마찬가지로 output[1]은 두 번째 샘플의 전체 시퀀스 출력임
  score = model.predict(encoded_input)[0][0]

  if score > 0.5 :
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [30]:
sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')

1/1 [==============================] - 4s 4s/step
96.89% 확률로 긍정 리뷰입니다.



In [31]:
sentiment_predict('이 영화 핵노잼 ㅠㅠ')

1/1 [==============================] - 0s 42ms/step
97.29% 확률로 부정 리뷰입니다.

